In [3]:
import numpy as np
from szar.counts import ClusterCosmology,Halo_MF
from szar.szproperties import SZ_Cluster_Model
from configparser import SafeConfigParser
from orphics.tools.io import dictFromSection,listFromConfig
import cPickle as pickle

home = '/Users/nab/Repos/SZ_filter/'

!pwd

iniFile = home+'input/pipeline.ini'
expName = 'CCATP-propv2'
gridName = 'grid-owl2'
cal = 'owl2'

Config = SafeConfigParser()
Config.optionxform=str
Config.read(iniFile)

fparams = {}
for (key, val) in Config.items('params'):
    if ',' in val:
        param, step = val.split(',')
        fparams[key] = float(param)
    else:
        fparams[key] = float(val)

bigDataDir = Config.get('general','bigDataDirectory')
clttfile = Config.get('general','clttfile')
constDict = dictFromSection(Config,'constants')
clusterDict = dictFromSection(Config,'cluster_params')
beam = listFromConfig(Config,expName,'beams')
noise = listFromConfig(Config,expName,'noises')
freq = listFromConfig(Config,expName,'freqs')
lknee = listFromConfig(Config,expName,'lknee')[0]
alpha = listFromConfig(Config,expName,'alpha')[0]

calFile = bigDataDir+"lensgrid_grid-"+cal+"_"+cal+".pkl"

version = Config.get('general','version')

mexp_edges, z_edges, lndM = pickle.load(open(calFile,"rb"))

mgrid,zgrid,siggrid = pickle.load(open(bigDataDir+"szgrid_"+expName+"_"+gridName+ "_v" + version+".pkl",'rb'))

assert np.all(mgrid==mexp_edges)
assert np.all(z_edges==zgrid)

#print lndM

cc = ClusterCosmology(fparams,constDict,clTTFixFile=clttfile)
HMF = Halo_MF(cc,mgrid,zgrid)

qs = listFromConfig(Config,'general','qbins')
qspacing = Config.get('general','qbins_spacing')
if qspacing=="log":
    qbin_edges = np.logspace(np.log10(qs[0]),np.log10(qs[1]),int(qs[2])+1)
elif qspacing=="linear":
    qbin_edges = np.linspace(qs[0],qs[1],int(qs[2])+1)
else:
    raise ValueError
    
SZProp = SZ_Cluster_Model(cc,clusterDict,rms_noises = noise,fwhms=beam,freqs=freq,lknee=lknee,alpha=alpha)

Mwl = 10**HMF.Mexp
z_arr = HMF.zarr
M_arr =  np.outer(HMF.M,np.ones([len(z_arr)]))

#if HMF.sigN is None: HMF.updateSigN(SZProp)
#sigN = HMF.sigN
#np.savetxt(home+'/tests/sigN_arr_save.txt',sigN)

sigN = np.loadtxt(home+'/tests/sigN_arr_save.txt')

#SZProp = SZ_Cluster_Model(cc,clusterDict,rms_noises = noise,fwhms=beam,freqs=freq,lknee=lknee,alpha=alpha)

q_arr = (qbin_edges[1:]+qbin_edges[:-1])/2.

i = 5
kk = 2
jj = 10

blah_orig = SZProp.P_of_qn(SZProp.lnY,M_arr[:,i],z_arr[i],sigN[:,i],q_arr[kk])
blah_mwl = SZProp.Mwl_prob (Mwl[jj],M_arr[:,i],lndM[:,i])
blah,blah2 = SZProp.P_of_qn_corr(SZProp.lnY,M_arr[:,i],z_arr[i],sigN[:,i],q_arr[kk],Mwl[jj],lndM[:,i])

#print blah_orig
#print blah_mwl

print blah_orig * blah_mwl
print blah, blah2

#dN_dmqz_corr = HMF.N_of_mqz_SZ_corr(lndM,qbin_edges,SZProp)                                                                  
#dN_dmqz = HMF.N_of_mqz_SZ(lndM,qbin_edges,SZProp)

/Users/nab/Repos/SZ_filter
((222, 130), (), (222,))
[[  7.07813258e-09   7.14425702e-09   7.21803433e-09 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  7.07443593e-09   7.14013461e-09   7.21343277e-09 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  7.07074030e-09   7.13601355e-09   7.20883300e-09 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 ..., 
 [  6.53453080e-09   6.54046133e-09   6.54702165e-09 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  6.53367034e-09   6.53950954e-09   6.54596874e-09 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  6.53282105e-09   6.53857011e-09   6.54492953e-09 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]]
No corr
((222, 130), (222, 130), (130,), (129,))
size 2
((222, 130), (), (222,), (222,))
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1. 

/Users/nab/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:17: DeprecationWarning: The SafeConfigParser class has been renamed to ConfigParser in Python 3.2. This alias will be removed in future versions. Use ConfigParser directly instead.
/Users/nab/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:19: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.


In [4]:
print blah - (blah_orig * blah_mwl)
print blah - blah2

[  2.42661816e-12   2.37289817e-12   2.32215617e-12   2.27441861e-12
   2.22968634e-12   2.18793817e-12   2.14913296e-12   2.11321024e-12
   2.08008947e-12   2.04966778e-12   2.02181608e-12   1.99637346e-12
   1.97313956e-12   1.95186469e-12   1.93223767e-12   1.91387115e-12
   1.89628473e-12   1.87888646e-12   1.86095420e-12   1.84161924e-12
   1.81988379e-12   1.79525480e-12   1.76788179e-12   1.73802024e-12
   1.70600636e-12   1.67225534e-12   1.63725833e-12   1.60157893e-12
   1.56585072e-12   1.53077759e-12   1.49711725e-12   1.46519782e-12
   1.43485912e-12   1.40589554e-12   1.37807079e-12   1.35111107e-12
   1.32469764e-12   1.29845920e-12   1.27196462e-12   1.24471745e-12
   1.21617045e-12   1.18610697e-12   1.15470343e-12   1.12219360e-12
   1.08885129e-12   1.05498800e-12   1.02095052e-12   9.87119072e-13
   9.53906801e-13   9.21761933e-13   8.91158567e-13   8.62261739e-13
   8.34893168e-13   8.08842519e-13   7.83875012e-13   7.59723520e-13
   7.36080003e-13   7.12586911e-13